<p>
    <font size="6"; face= "Trebuchet MS"; color="green">
            <b>
                This notebook is created by: <a href="https://github.com/mahdi-khoshmaram">Mahdi khoshmaram</a> 
            </b>
</p>

<p>
    <font size="5"; face= "Trebuchet MS"; color="brown">
            <b>
             LSH algorithm from scratch!  
            </b>
    </font>
</p>
<p>
    <font size="3"; face= "Trebuchet MS"; color="brown">
            <b>
             The whole code is written by myself and I do not use any helper packages except <i>Numpy<i>, <i>Pandas<i>, <i>Random<i> and <i>tqdm<i>.
            </b>
    </font>
</p>



## Import numpy, pandas, random and tqdm modules

import pandas as pd
import numpy as np
import random
from tqdm import tqdm

In [11]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import random

## Read csv file
I set **doc_id** column as index column.

In [4]:
df = pd.read_csv("docs_for_lsh.csv", index_col='doc_id')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000000 entries, 0 to 999999
Columns: 200 entries, 0 to 199
dtypes: int64(200)
memory usage: 1.5 GB


In [6]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
doc_id,,,,,,,,,,,,,,,,,,,,,
0,1,0,1,0,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,1,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,1,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


## Setting parameter of my universal minhash function.
I want to have a signature of length **25** for each document, so we should have 25 minhash functions! 

you can generate family of hash functions using following simple equation:


$$
h(x,a,b) = ((ax+b) \mod p) \mod m
$$



- **x** is key you want to hash
- **a** is any odd number you can choose between 1 to p-1 inclusive.
- **b** is any number you can choose between 0 to p-1 inclusive.
- **p** is a prime number that is greater than max possible value of x
- **m** is a max possible value you want for hash code + 1

In [7]:
# p is a prime number that is greater than max possible value of x (211 is the first prime number after 200)
p = 211

# m is a max possible value you want for hash code + 1 (200+1)
m = 201

# length of my signatures
num_minhash = 25

# params = list of (a,b)
params = []

# I made this loop to create 25 (a, b) which satisfies that a is odd and both a & b are less than p-1
while len(params) != 25:
    q = random.randint(1,p)
    w = random.randint(0,p)
    if (q%2 != 0) and (q,w) not in params:
        params.append((q,w))

## Defining Minhash function 

<i>my Pseudocode for Minhash function<i>

```python
Define signature matrix with shape of (number_of_document, number_minhash_functions) = (1000000, 25)
for each column in signature_matrix:
    for each element in params(a,b):
        for row of rows in dataframe:
            calculate min_hash value with parameters a,b,p,m and row id
            find 1s in that row and their indexes
            in signature_matrix substitude hash_value with values in signature matrix  if hash_value < current value
```

#### objective for creating update function

To implement bellow part in pseudocode:
```python
in signature_matrix substitude hash_value with values in signature matrix  if hash_value < current value
```

In [8]:
def update(matrix, col_of_matrix, index_sequence, value):
    for index in index_sequence:
        if matrix[index, col_of_matrix] > value:
            matrix[index, col_of_matrix] = value
    return matrix

In [9]:
def min_hash(df, params, num_minhash):
    num_document = df.shape[0]
    sig_matrix = np.ones((num_document,num_minhash)) * np.inf
    sig_col = 0
    for a,b in params:
        print(f"hash function {sig_col + 1} is computing...")
        for i in tqdm(range(df.shape[1]-1)):
            hash_val = ( ( a * i + b ) % p ) % m
            indices_of_1 = [index for index, value in enumerate(df.iloc[:,i].values) if value == 1]
            update(sig_matrix, sig_col, indices_of_1, hash_val)

        sig_col += 1
        if sig_col > 25:
            break
    return sig_matrix

## calculating Signatures 

In [12]:
signature = min_hash(df, params, 25)

hash function 1 is computing...


100%|███████████████████████████████████| 199/199 [00:31<00:00,  6.23it/s]


hash function 2 is computing...


100%|███████████████████████████████████| 199/199 [00:32<00:00,  6.09it/s]


hash function 3 is computing...


...

100%|███████████████████████████████████| 199/199 [00:35<00:00,  5.65it/s]


hash function 24 is computing...


100%|███████████████████████████████████| 199/199 [00:35<00:00,  5.59it/s]


hash function 25 is computing...


100%|███████████████████████████████████| 199/199 [00:35<00:00,  5.64it/s]


## Save signature matrix
Due to long time of processing, I save signature_matrix in **.npy** Format.

In [13]:
np.save('signature',signature)

## load matrix form file in .npy format

In [21]:
# signature = np.load('signature.npy')

## Transpose signature matrix

signature matrix rows are documents and columns are signature values. 
on the other hand, Refrence, it is opposite, meaning rows are signature values and columns are documents. So I transpose the matrix.

In [19]:
signature.T.shape

(25, 1000000)

## Implementing LSH algorithm 

for **LSH** the number of bands I decided to use is 25.

Also for bucket hashing I decide a linear function which avoids of **False positives** because of linearity characteristic! 

$$
h(X1,X2,X3,X4,X5) = X1*c1 + X2*c2 + X3*c3 + X4*c4 + X5*c5 
$$

* Here **X** is elements of signature matrix
* [ c1, c2, c3, c4, c5 ] = [2, 4, 1, 5, 7]

In [20]:
band1 = signature.T[0:5,:]
band2 = signature.T[6:10,:]
band3 = signature.T[11:15,:]
band4 = signature.T[16:20,:]
band5 = signature.T[20:24,:]

band_list = [band1, band2, band3, band4, band5]
hash_param = [2,4,1,5,7]

Now:
- We have 5 buckets. each bucket is saved in a dictionary named **bucket** 
- And all dictionaries are saved in a list named **buckets**

I hash each band of each document to a bucket number. 
After calculating bucket number, a save becket number as ***key*** and ***dic_id*** as a value in bucket dictinary.

After finishing each bucket, I append current bucket dictionary to *bucket list* and then reset bucket dictionary.

In [21]:
bucket = {}
buckets = []
for band in band_list:
    for document in range(band.shape[1]):
        hash_val = np.dot(band1[:,document],[2,4,1,5,7])
        if hash_val in bucket.keys():
            bucket[hash_val].append(document)
        else:
            bucket[hash_val]=[document]
    buckets.append(bucket)
    bucket = {}

In [22]:
len(buckets)

5

## Defining jacard function 

For finding two rows of two documents that both of them has 1 value, I used ***AND*** operation
For finding two rows of two documents that at least one of them  has 1 value, I used ***OR*** operation. 


I assign:
- **similar** = len(and_output)
- **total** = len(or_output)



Then jacard value will be:
$$
jacard = similar / total
$$

In [23]:
def jacard(a,b):
    sim = len([index for index, val in enumerate(a & b) if val == 1])
    total = len([index for index, val in enumerate(a | b) if val == 1])
    return sim/total

## finding documents similar to domcument-0

In this cell, I just do some filter:

In [24]:
k = 1
sim_to_doc0_docs = set()
for bucket in buckets:
    for hash_value in bucket.keys():
        if (len(bucket[hash_value]) > 1) and (0 in bucket[hash_value]):
            for doc in bucket[hash_value]:
                if doc != 0:
                    sim_to_doc0_docs.add(doc)
    k += 1

#### number of all documents similar to document_0

In [25]:
len(sim_to_doc0_docs)

2974

#### assign variable to document_0

In [26]:
doc_0 = df.iloc[0,:].values

## Calculating jacard values for ducuments that are recognized as similar by LSH algorithm 

In [27]:
jacard_eval = {}
for doc in sim_to_doc0_docs:
    document = df.iloc[doc,:].values
    j = jacard(doc_0,document)
    jacard_eval[doc] = j

#### Sort by High jacard values

In [28]:
tmp = list()
for k,v in jacard_eval.items():
    tmp.append((v,k))
tmp = sorted(tmp, reverse = True)

## Picking top 30 document

In [29]:
for jacard, doc in tmp[0:30]:
    print(f'(0,{doc}) ---> {jacard}')

(0,84306) ---> 0.8333333333333334
(0,81480) ---> 0.8333333333333334
(0,40298) ---> 0.8333333333333334
(0,20854) ---> 0.8333333333333334
(0,84520) ---> 0.8181818181818182
(0,81289) ---> 0.8181818181818182
(0,72156) ---> 0.8181818181818182
(0,68730) ---> 0.8181818181818182
(0,58694) ---> 0.8181818181818182
(0,52076) ---> 0.8181818181818182
(0,48131) ---> 0.8181818181818182
(0,26980) ---> 0.8181818181818182
(0,23585) ---> 0.8181818181818182
(0,2575) ---> 0.8181818181818182
(0,1331) ---> 0.8181818181818182
(0,89580) ---> 0.75
(0,83373) ---> 0.75
(0,73687) ---> 0.75
(0,72078) ---> 0.75
(0,70964) ---> 0.75
(0,68114) ---> 0.75
(0,63678) ---> 0.75
(0,62869) ---> 0.75
(0,58819) ---> 0.75
(0,57514) ---> 0.75
(0,57005) ---> 0.75
(0,49989) ---> 0.75
(0,47558) ---> 0.75
(0,45793) ---> 0.75
(0,42302) ---> 0.75
